Experimental space

In [ ]:
%pip install openai openai-agents python-dotenv pydantic GitPython PyGithub

In [1]:
from git import Repo
from github import Github
from pydantic import BaseModel, Field
import asyncio
from dataclasses import dataclass
import os
from dotenv import load_dotenv

load_dotenv()
github_token = os.getenv("GITHUB_TOKEN")
openai_key = os.getenv("OPENAI_API_KEY")

In [2]:
notebook_path = os.getcwd()
print(notebook_path)


/Users/Prashant/LOCAL/Data/github/python/openai-git-agent


In [ ]:
@dataclass
class CodeTask(BaseModel):
    """Schema for code solution"""
    code: str = Field(..., title="Code solution")
    task_description: str = Field(..., title="Description of task")

In [3]:
@dataclass
class CodeSolution(BaseModel):
    """Schema for code solution"""
    code: str = Field(..., title="Code solution")
    description: str = Field(..., title="Description of the code solution")

In [38]:
@dataclass
class AgentTaskData(BaseModel):
    task_content: str = Field(..., title="Description of task to be performed")
    repo_dir: str = Field(..., title="Repository directory")
    generation: CodeSolution | None = Field(..., title="Code solution")
    iterations: int = Field(..., title="Number of iterations")
    status: bool = Field(..., title="Status of the task")


In [4]:
@dataclass
class CodeFeedback(BaseModel):
    feedback: str = Field(..., title="Feedback on code solution")
    refine_further: bool = Field(..., title="Flag to determine if code solution needs further refinement")

In [5]:
from agents import Agent, InputGuardrail,GuardrailFunctionOutput, Runner, ItemHelpers, TResponseInputItem, trace
from pydantic import BaseModel
import asyncio

python_coder_agent = Agent(
    model="o3-mini",
    name="Coder",
    handoff_description="Specialist agent for writing python code and test cases",
    instructions="You are a expert software engineer coding in python, who follows best practices and writes testable code. Write code based on task description by user.",
    output_type=CodeSolution,
)
reviewer_agent = Agent(
    model="o3-mini",
    name="Reviewer",
    handoff_description="Specialist agent for reviewing python code, ensures that code is well written and follows best practices",
    instructions="You are a expert code reviewer that reviews code in python. Validate that code is fully implemented, it follows user requirement, and provides feedback for correction and improvement.",
    output_type=CodeFeedback,
)

In [ ]:
from github import Github
g = Github(github_token)
for local_repo in g.get_user().get_repos():
    print(local_repo.name)

aws-code-pipeline-spring-hello
Blockchain_hackathon
cassandra-developers
CKAD-Practice-Questions
coding-challenges
conductor
ContextFreeGrammarNLP
docker-kubernetes
docker-netflix-conductor
file-reader-writer-maven-plugin
food-funda-spring-cloud-functions
genai-scratch-repo
git_repo
interview-guide
intuit-council-service
java-gpt4o-hello
java-spring-symphony-bot
java8lambda
kafka-trending-news
MITMissedSemester
ml-regression-models
mlops-zenml-customer-satisfaction-prediction
money-transfer-service
openai-llmagent-diet-plan
openrewrite-custom-mvn-profile
prazjain
py-dev-tools
python-cheat-sheet
python-dockerized-server
python-flask-rest-api
python-mlflow-basics
python-multiple-pdf-chat
python-NLP-ML
python-zenml-intro
qs-java-boot-junit-cucumber
rasa-hello-bot
searchengine
siri-search-service
sonar-java-detect-annotation
spring-boot-swagger
spring-cloud-coordinating-services
springboot-docker-k8s
springboot-kafka-docker
springboot-prometheus-grafana
streamlit-calculator-app
terraform-a

In [25]:
print('before init github')
g = Github(github_token)
print('after init github')
py_repo = g.get_repo('prazjain/genai-scratch-repo')

before init github
after init github


In [42]:
py_branch = py_repo.get_branch('python/genai-agent-task')
print('after get repo')
print('Repo name : ' , py_repo.name)
print('Repo url : ' , py_repo.url)
print('Repo clone url : ' , py_repo.clone_url)
print('Default branch : ' , py_repo.default_branch)
print('Branch name : ' , py_branch.name)
print('Branch commit sha : ' , py_branch.commit.sha)
print('Branch commit url : ' , py_branch.commit.url)

after get repo
Repo name :  genai-scratch-repo
Repo url :  https://api.github.com/repos/prazjain/genai-scratch-repo
Repo clone url :  https://github.com/prazjain/genai-scratch-repo.git
Default branch :  main
Branch name :  python/genai-agent-task
Branch commit sha :  d0896c989a7f04d63d8ebd8f1675da649f5b482d
Branch commit url :  https://api.github.com/repos/prazjain/genai-scratch-repo/commits/d0896c989a7f04d63d8ebd8f1675da649f5b482d


In [31]:
print('Repo clone url : ' , py_repo.clone_url)
repo_dir = os.path.join(notebook_path, 'agent-task')
local_repo = Repo.clone_from(py_repo.clone_url, repo_dir, branch='python/genai-agent-task')


Repo clone url :  https://github.com/prazjain/genai-scratch-repo.git


In [32]:
#switch to the branch python/genai-agent-task
#local_repo.git.checkout(py_branch.name)
print('after clone repo')
#print(local_repo)
print(local_repo.git.status())
print(local_repo.git.branch())
#print(local_repo.git.log())

after clone repo
On branch python/genai-agent-task
Your branch is up to date with 'origin/python/genai-agent-task'.

nothing to commit, working tree clean
* python/genai-agent-task


In [6]:
def clone_repo(github_token: str, repo_url: str, repo_dir: str='') -> tuple[str, str]:
    """Clone a repo from github"""
    try:
        if not repo_dir:
            #script_dir = os.path.dirname(os.path.abspath(notebook_path))
            #repo_dir = os.path.join(script_dir, 'agent-task')
            repo_dir = os.path.join(notebook_path, 'agent-task')

        if os.path.exists(repo_dir):
            print(f"A repo already exists at {repo_dir}, deleting it before cloning...")
            os.system(f"rm -rf {repo_dir}")

        print('before init github')
        g = Github(github_token)
        print('after init github')
        py_repo = g.get_repo(repo_url)
        py_branch = py_repo.get_branch('python/genai-agent-task')
        print('after get repo')
        print('Repo name : ' , py_repo.name)
        print('Repo url : ' , py_repo.url)
        print('Repo clone url : ' , py_repo.clone_url)
        local_repo = Repo.clone_from(py_repo.clone_url, repo_dir)
        #switch to the branch python/genai-agent-task
        local_repo.git.checkout(py_branch.commit.sha)
        print('after clone')
        return repo_dir, ""
    except Exception as e:
        return "", f"Repository setup failed: {str(e)}"



In [27]:
#result = clone_repo(github_token, 'https://github.com/prazjain/genai-scratch-repo')
result = clone_repo(github_token, 'prazjain/genai-scratch-repo')
print(result)

before init github
after init github
after get repo
Repo name :  genai-scratch-repo
Repo url :  https://api.github.com/repos/prazjain/genai-scratch-repo
Repo clone url :  https://github.com/prazjain/genai-scratch-repo.git
after clone
('/Users/Prashant/LOCAL/Data/github/python/openai-git-agent/agent-task', '')


In [8]:
def read_task_file(repo_dir: str) -> tuple[str, str]:
    """Read the task markdown file."""
    try:
        #task_path = os.path.join(repo_dir, 'tasks', 'task.md')
        task_path = os.path.join(repo_dir, 'task.md')
        with open(task_path, 'r') as f:
            return f.read(), ""
    except Exception as e:
        return "", f"Failed to read task: {str(e)}"

In [17]:
repo_dir = result[0]
task_content, error = read_task_file(repo_dir)
print(task_content)
print(error)

## Array Product Calculator  

### Problem Description  

Create a function that takes a list of integers and returns a new list where each element at position i is the product of all original numbers except the one at position i.  

### Requirements  
* Function name shoud be `calculate_products`  
* Input: List of integers  
* Output: List of integers  
* Handle empty list and single element list cases  
* Handle lists with zeros
* File name should be `array_products.py`

  ### Examples
  Input: [1, 2, 3, 4]
  Output: [24, 12, 8, 6]

  Input: [1, 0, 3, 4]
  Output: [0, 12, 0, 0]


  ### Acceptance Criteria
  * Must include type hints
  * Must include docstring with examples
  * Must include error handling for invalid inputs
  




In [15]:
async def generate_solution() -> CodeSolution:
    """Generate code solution based on task description."""

    max_iterations = 3

    coder_convo_history = []
    reviewer_convo_history = []

    coder_convo_history.append({"role": "user", 
                                "content": task_content })
    reviewer_convo_history.append({"role": "user",
                                   "content": f"Task description : \n{task_content}",
                             })
    print("Generating code ...")
    code_solution = await Runner.run(python_coder_agent, coder_convo_history)
    #.final_output_as(CodeSolution)
    print("Generated code.")
    #print('Lets check code final output ...')
    #print(code_solution.final_output.code)
    #print(code_solution.final_output.description)
    #print(code_solution.code)
    for i in range(1, max_iterations+1):
        print(f"Iteration {i} of feedback..")
        reviewer_convo_history.append({"role": "user",
                                       "content": f"Please provide feedback on code solution {i} : \n{code_solution.final_output.code} ",
                                 })
        code_feedback = await Runner.run(reviewer_agent, reviewer_convo_history)
        #.final_output_as(CodeFeedback)
        print("Received feedback")
        if code_feedback.final_output.refine_further:
            print("Need further refinement.")
            coder_convo_history.append({"role": "user",
                                        "content": f"Please improve the code based on feedback : \n{code_feedback.final_output.feedback} "})
            print("Refining code ...")
            code_solution = await Runner.run(python_coder_agent, coder_convo_history)
            #.final_output_as(CodeSolution)
            print("Refined code.")
        else:
            print("Don't need further refinement.")
            break
    return code_solution


In [18]:
sol = await generate_solution()
print("---" * 50)
print(sol)
print("---" * 50)
print(sol.final_output.code)

Generating code ...
Generated code.
Iteration 1 of feedback..
Received feedback
Don't need further refinement.
------------------------------------------------------------------------------------------------------------------------------------------------------
RunResult:
- Last agent: Agent(name="Coder", ...)
- Final output (CodeSolution):
    {
      "code": "from typing import List\n\n\ndef calculate_products(numbers: List[int]) -> List[int]:\n    \"\"\"\n    Given a list of integers, return a new list such that each element at index i is\n    the product of all numbers in the original list except the one at i.\n\n    Examples:\n    >>> calculate_products([1, 2, 3, 4])\n    [24, 12, 8, 6]\n\n    >>> calculate_products([1, 0, 3, 4])\n    [0, 12, 0, 0]\n\n    >>> calculate_products([])\n    []\n\n    >>> calculate_products([5])\n    [1]\n    \n    Args:\n        numbers (List[int]): List of integers.\n\n    Returns:\n        List[int]: A list where each element is the product of all n

In [ ]:
def test_solution(task_data: AgentTaskData, solution: CodeSolution) -> bool:
    """Test the code solution"""
    return True

In [28]:
def create_pr(solution: CodeSolution, branch, iterations) -> bool:
    """Create a pull request"""
    try:
        print('Creating PR ...')
        #print('Solution : ', solution)
        sol = solution.code
        #print(sol)
        local_repo = Repo(repo_dir)
        print('Repo : ', local_repo)

        # Local git operations
        branch_name = f"{branch}-agent-solution-{iterations}"
        current = local_repo.create_head(branch_name)
        current.checkout()
        with open(os.path.join(repo_dir, 'solution.py'), 'w') as f:
            f.write(sol)
        local_repo.index.add(['solution.py'])
        local_repo.index.commit("Agent solution")
        local_repo.git.push('origin', branch_name)

        # remote git 
        g = Github(github_token)
        py_repo = g.get_repo('prazjain/genai-scratch-repo')

        # Github operations to create PR
        pr = py_repo.create_pull(
            title="Add Array Products Calculator",
            body=f"Implements array products calculator with the following approach:\n\n{solution.description}",
            base=branch,
            head=branch_name
        )
        print(f"Created PR: {pr.html_url}")
        return True
    except Exception as e:
        print(f"Failed to create PR: {str(e)}")
        return False

In [29]:
status = create_pr(sol.final_output, 'python/genai-agent-task', 1)
print(status)

Creating PR ...
Repo :  <git.repo.base.Repo '/Users/Prashant/LOCAL/Data/github/python/openai-git-agent/agent-task/.git'>
Created PR: https://github.com/prazjain/genai-scratch-repo/pull/1
True
